# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121862e+02     1.589379e+00
 * time: 0.6774599552154541
     1     1.057670e+01     9.392203e-01
 * time: 1.8896818161010742
     2    -1.197663e+01     1.021141e+00
 * time: 2.4454538822174072
     3    -3.404865e+01     7.766549e-01
 * time: 3.2689919471740723
     4    -4.761936e+01     6.074254e-01
 * time: 4.093204021453857
     5    -5.693613e+01     2.031675e-01
 * time: 4.8945558071136475
     6    -5.983430e+01     1.580816e-01
 * time: 5.436577796936035
     7    -6.097309e+01     6.572126e-02
 * time: 5.994704008102417
     8    -6.141990e+01     5.480991e-02
 * time: 6.539752960205078
     9    -6.166372e+01     5.226871e-02
 * time: 7.084851026535034
    10    -6.186608e+01     2.535829e-02
 * time: 7.634469985961914
    11    -6.200132e+01     1.974553e-02
 * time: 8.180641889572144
    12    -6.206913e+01     1.535799e-02
 * time: 8.72635293006897
    13    -6.212287e+01     1.210669e-02
 * time: 9.272773027420044
   

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671066
    AtomicLocal         -18.8557659
    AtomicNonlocal      14.8522629
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336820

    total               -62.261666460738